In [36]:
import json
import csv
from collections import defaultdict, OrderedDict
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt


In [37]:
DATA_ROOT = '/var/data/processed'
GOLDEN_ROOT = os.path.join(DATA_ROOT, 'golden')
CSV_DATA_ENTRY = os.path.join(GOLDEN_ROOT, 'Data_Entry_Golden.csv')
CSV_DIRECTORY_REFERENCE = os.path.join(GOLDEN_ROOT, 'randomized_lut.csv')
CSV_DIRECTORY_FILLED = os.path.join(GOLDEN_ROOT, 'filled','RandomizedImages1.filled100.csv')
ORIGINAL_IMAGES_DIR = '/var/data/original/data/images'
GOLDEN_IMAGES_DIR = os.path.join(GOLDEN_ROOT, 'images')
GOLDEN100_CSV_DIR = os.path.join(GOLDEN_ROOT, 'golden100.csv')
GOLDEN100__JSON_DIR = os.path.join(GOLDEN_ROOT, 'golden100.json')
GOLDEN100_SOURCE__JSON_DIR = os.path.join(GOLDEN_ROOT, 'golden_data.json')

In [4]:
EXPT_NAME = 'expt_001'
EXPT_DIR = os.path.join(DATA_ROOT, EXPT_NAME, 'data')
EMBEDDINGS_DIR = os.path.join(EXPT_DIR, 'embeddings')

In [38]:
golden_documents = []
json_files = [pos_json for pos_json in os.listdir(EMBEDDINGS_DIR) if pos_json.endswith('.json')]
for file in json_files:
    path = os.path.join(EMBEDDINGS_DIR, file)
    with open(path) as f:
        doc = json.load(f)
        if (doc['subset'] == 'golden'):
            golden_documents.append(doc)

val_documents = []
json_files = [pos_json for pos_json in os.listdir(EMBEDDINGS_DIR) if pos_json.endswith('.json')]
for file in json_files:
    path = os.path.join(EMBEDDINGS_DIR, file)
    with open(path) as f:
        doc = json.load(f)
        if (doc['subset'] == 'golden'):
            val_documents.append(doc)            

'''for doc in documents:
    doc['mu'] = np.asarray(doc['mu'])
    doc['sigma'] = np.asarray(doc['sigma'])
    doc['labels'] = np.asarray(doc['labels'])'''

"for doc in documents:\n    doc['mu'] = np.asarray(doc['mu'])\n    doc['sigma'] = np.asarray(doc['sigma'])\n    doc['labels'] = np.asarray(doc['labels'])"

# Evaluations

In [31]:
#create a hashtable mapping source filenames to source labels.

csvfile =  open(CSV_DATA_ENTRY) 
filename_to_source_labels = defaultdict(list)
row_count = sum(1 for row in csv.reader(csvfile))
csvfile =  open(CSV_DATA_ENTRY) 

LABEL_KEYS = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Effusion",
    "Emphysema",
    "Fibrosis",
    "Hernia",
    "Infiltration",
    "Mass",
    "Nodule",
    "No Finding",
    "Pleural_Thickening",
    "Pneumonia",
    "Pneumothorax",
]

nb_label_keys = len(LABEL_KEYS)
label_keys_by_id = LABEL_KEYS
ids_by_label_key = OrderedDict([(v, k) for k, v in enumerate(label_keys_by_id)])

# Iterate through file; represent labels as N-hot binary lists
for i,line in enumerate(csvfile.readlines()[1:row_count]):
    
    tokens = line.strip().split(',')
    labels = []

    for key in tokens[1].split('|'):
        labels.append(key)
    
    filename_to_source_labels[tokens[0]].append(labels)   
    
def getlabels(onehot_labels):
    list = []
    for index_value, index in enumerate(onehot_labels):
        if(index == 1):
            list.append(ids_by_label_key.get(index_value))

    return list


In [27]:
'''from collections import OrderedDict
LABEL_KEYS2 = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Effusion",
    "Emphysema",
    "Fibrosis",
    "Hernia",
    "Infiltration",
    "Mass",
    "Nodule",
    "Pleural_Thickening",
    "Pneumonia",
    "Pneumothorax",
    "No Finding", 
]

label_keys_by_id = LABEL_KEYS2
ids_by_label_key2 = OrderedDict([(k,v) for k, v in enumerate(label_keys_by_id)])
  
def getlabels(onehot_labels):
    list = []
    for index_value, index in enumerate(onehot_labels):
        if(index == 1):
            list.append(ids_by_label_key2.get(index_value))

    return list'''

'from collections import OrderedDict\nLABEL_KEYS2 = [\n    "Atelectasis",\n    "Cardiomegaly",\n    "Consolidation",\n    "Edema",\n    "Effusion",\n    "Emphysema",\n    "Fibrosis",\n    "Hernia",\n    "Infiltration",\n    "Mass",\n    "Nodule",\n    "Pleural_Thickening",\n    "Pneumonia",\n    "Pneumothorax",\n    "No Finding", \n]\n\nlabel_keys_by_id = LABEL_KEYS2\nids_by_label_key2 = OrderedDict([(k,v) for k, v in enumerate(label_keys_by_id)])\n  \ndef getlabels(onehot_labels):\n    list = []\n    for index_value, index in enumerate(onehot_labels):\n        if(index == 1):\n            list.append(ids_by_label_key2.get(index_value))\n\n    return list'

In [7]:
#create a hash table mapping id numbers to filenames for 100 golden data
csvfile =  open(CSV_DIRECTORY_REFERENCE) 
index = defaultdict(list)
for line in csvfile.readlines():
    tokens = line.strip().split(',')
    index[tokens[0]].append(tokens[1])

In [12]:
#create a new csv file for the 100 golden data out of 400 csv file

filenames_golden = []

i = 0
for value in index.values():
    if(i != 100):
        filenames_golden.append(value[0])
        i+=1
    else:
        break
    
csvfile_read =  open(CSV_DATA_ENTRY) 


csvfile_write =  open(GOLDEN100_CSV_DIR, 'w') 
write_lines = ""


for i,line in enumerate(csvfile_read.readlines()):

    if(i==0):
        csvfile_write.write(line)

    tokens = line.strip().split(',')
    
    if tokens[0] in filenames_golden:
        csvfile_write.write(line)


In [19]:
#create a hash table mapping filenames to radiologist labels. 
#create json for radiologist id, filename, labels, and other  

csvfile =  open(CSV_DIRECTORY_FILLED) 
radiologist_csvtokens = []
row_count = sum(1 for row in csv.reader(csvfile))
csvfile =  open(CSV_DIRECTORY_FILLED) 
filename_to_radiologist_labels = defaultdict(list)
filename_to_radiologist_labels_onehot = defaultdict(list)


# Iterate through file; represent labels as N-hot binary lists
for i,line in enumerate(csvfile.readlines()[1:(row_count-2)]):
    
    tokens = line.strip().split(',')
    onehot_labels = np.zeros(15).astype(int)
    
    for i,key in enumerate(tokens[1:12]):
        if(key == 'x'):
            onehot_labels[i] = 1
    if(tokens[12] == 'x'):
        onehot_labels[12] = 1
    if(tokens[13] == 'x'):
        onehot_labels[13] = 1
    if(tokens[14] == 'x'):
        onehot_labels[14] = 1
    if(tokens[15] == 'x'):
        onehot_labels[11] = 1
    
    filename_to_radiologist_labels[index[tokens[0]][0]].append(getlabels(onehot_labels))
    filename_to_radiologist_labels_onehot[index[tokens[0]][0]].append(onehot_labels)
    
    radiologist_csvtokens.append({
        'id': tokens[0],
        'filename': index[tokens[0]],
        'labels': getlabels(onehot_labels),
        'other': tokens[16]
    })


In [34]:
#generate the golden100_source.json file based on the labels we got from the source

csvfile_read =  open(GOLDEN100_CSV_DIR, 'r') 
json_lines = []

csvfile_read.readline()
for line in enumerate(csvfile_read.readlines()):

    tokens = line[1].strip().split(',')
    labels = [0]*nb_label_keys
    
    for key in tokens[1].split('|'):
        labels[ids_by_label_key[key]] = 1
            
    json_lines.append({
        'filename': tokens[0],
        'labels': labels,
        'age':tokens[4],
        'gender': tokens[5],
        'view': tokens[6],
        'dx': float(tokens[9]),
        'dy': float(tokens[10]),
        'w': int(tokens[7]),
        'h': int(tokens[8]),
    })

        
with open(GOLDEN100__JSON_DIR, 'w') as outfile:
        json.dump(json_lines, outfile, separators=(',', ':'), indent = 2)

In [ ]:
#generate the golden100.json file based on the labels we got from the source

csvfile_read =  open(GOLDEN100_CSV_DIR, 'r') 
json_lines = []
labels = []
csvfile_read.readline()
for line in enumerate(csvfile_read.readlines()):

    tokens = line[1].strip().split(',')
    json_lines.append({
        'filename': tokens[0],
        'labels': filename_to_radiologist_labels_onehot[tokens[0]][0].tolist(),
        'age':tokens[4],
        'gender': tokens[5],
        'view': tokens[6],
        'dx': float(tokens[9]),
        'dy': float(tokens[10]),
        'w': int(tokens[7]),
        'h': int(tokens[8]),
    })

        
with open(GOLDEN100_SOURCE__JSON_DIR, 'w') as outfile:
        json.dump(json_lines, outfile, separators=(',', ':'), indent = 2)

In [28]:
#Comparison between radiologist and source labels.

dash = '-' * 120
print("id{:<10}  Radiologist{:<57} Source{:<20}"
          .format("",
                  "",
                  "",
                 ""))
print(dash + "\n")

for radiologist_item in radiologist_csvtokens:
    
    radiologist_labels = ""
    for x in radiologist_item['labels']:
        radiologist_labels += x +","
    
    labels = filename_to_source_labels[radiologist_item['filename'][0]][0]
    source_labels = ""
    for x in labels:
        source_labels += x +","
    
    print("{:<10s}  {:<57s} {:<30s}"
          .format(radiologist_item['id'],
                  radiologist_labels,
                  source_labels))

id            Radiologist                                                          Source                    
------------------------------------------------------------------------------------------------------------------------

000001      No Finding,                                               No Finding,                   
000002      Atelectasis,Effusion,Hernia,Pleural_Thickening,           Fibrosis,Hernia,              
000003                                                                Atelectasis,                  
000004      No Finding,                                               Nodule,                       
000005      No Finding,                                               No Finding,                   
000006                                                                No Finding,                   
000007      Consolidation,Infiltration,                               Effusion,Infiltration,        
000008      No Finding,                                      

In [29]:
def total_jaccard():
    intersection_count = 0
    union_count = 0
    for radiologist_item in radiologist_csvtokens:
        source_labels = filename_to_source_labels[radiologist_item['filename'][0]][0]
        radiologist_labels = radiologist_item['labels']
        intersection = set(source_labels) & set(radiologist_labels)
        union = set(source_labels) | set(radiologist_labels)
        intersection_count += len(intersection)
        union_count += len(union)
    return intersection_count/union_count

In [30]:
total_jaccard()

0.23770491803278687

In [31]:
#save golden images

images = []

for doc in radiologist_csvtokens:
    filename = doc['filename'][0]
    file_path = os.path.join(ORIGINAL_IMAGES_DIR, filename)
    img = mpimg.imread(file_path)
    save_path = os.path.join(GOLDEN_IMAGES_DIR, filename)
    plt.imsave(save_path, img)


In [138]:
confusion_matrix = np.zeros((15,15), dtype=int)
for doc in golden_documents:
    
    radiologist_labels = filename_to_radiologist_labels[doc['filename']][0]
    source_labels = filename_to_source_labels[doc['filename']][0]
    for r_label in radiologist_labels:
        i = ids_by_label_key1[r_label]
        if(r_label in source_labels):
            confusion_matrix[i][i] += 1 
        else:
            for s_label in source_labels:
                j = ids_by_label_key1[s_label]
                confusion_matrix[i][j] += 1 
    
    for s_label in source_labels:
        j = ids_by_label_key1[s_label]
        if(s_label in radiologist_labels):
            continue 
        else:
            for r_label in radiologist_labels:
                i = ids_by_label_key1[r_label]
                confusion_matrix[i][j] += 1 

In [137]:
print("{:<4s} ".format(""), end="")
for label_name in LABEL_KEYS1:
    print("{:<4s} ".format(label_name[:2]), end="")
print("\n")   
for i,row in enumerate(confusion_matrix):
    print("{:<4s} ".format(LABEL_KEYS1[i][:2]), end="")
    for j in row:
        print("{:<4d} ".format(j), end="")
    print("\n")    
    
    LABEL_KEYS1 = [
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Effusion",
    "Emphysema",
    "Fibrosis",
    "Hernia",
    "Infiltration",
    "Mass",
    "Nodule",
    "No Finding",
    "Pleural_Thickening",
    "Pneumonia",
    "Pneumothorax",
]

     At   Ca   Co   Ed   Ef   Em   Fi   He   In   Ma   No   No   Pl   Pn   Pn   

At   10   0    2    0    4    5    2    1    8    1    1    18   5    0    2    

Ca   3    0    2    1    0    0    0    0    2    0    0    12   0    0    0    

Co   5    0    1    0    3    4    0    0    8    0    2    8    1    0    4    

Ed   3    0    0    2    1    0    0    0    2    2    2    8    0    0    2    

Ef   4    0    1    0    5    10   2    1    5    2    3    18   3    0    5    

Em   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    

Fi   1    0    0    2    1    0    0    0    0    0    0    4    0    0    0    

He   1    0    0    0    0    0    1    1    0    0    0    2    0    0    0    

In   0    0    1    1    2    2    0    0    6    0    6    4    1    0    2    

Ma   0    0    0    0    2    0    0    0    2    2    3    6    2    0    0    

No   0    0    0    0    1    0    0    0    1    0    1    0    0    0    0    

No   2    0    0

# Classifier


In [129]:
def jaccard(item1, item2):
    
    intersection = set(item1) & set(item2)
    union = set(item1) | set(item2)
    return len(intersection)/len(union)

In [130]:
y = []
for doc in golden_documents:
    radiologist_labels = filename_to_radiologist_labels[doc['filename']][0]
    source_labels = filename_to_source_labels[doc['filename']][0]
    j = jaccard(radiologist_labels, source_labels)
    y.append(j)

In [133]:
X = []
for doc in golden_documents:
    cat = doc['mu']+doc['labels']
    X.append(cat)

In [143]:
val = []
for doc in val_documents:
    val.append( doc['mu'] + doc['labels'])

In [144]:
from sklearn.svm import SVR
import numpy as np

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(val)
y_lin = svr_lin.fit(X, y).predict(val)
y_poly = svr_poly.fit(X, y).predict(val)

ranking_rbf = sorted(y_rbf)
ranking_lin = sorted(y_lin)
ranking_poly = sorted(y_poly)

# Visualisation

In [198]:
from skimage.color import rgb2gray

#generate a sprite image consisting of all the images
def create_sprite_image(documents, directory):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    
    images = []
    for doc in documents:
        path = os.path.join(ORIGINAL_IMAGES_DIR, doc['filename'])
        img = mpimg.imread(path)
        if len(np.shape(img)) == 3:
            img = rgb2gray(img)
        
        images.append(img)

    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    
    
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))
    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
                
    plt.imsave(directory, spriteimage, cmap='gray')
    plt.imshow(spriteimage,cmap='gray')
    
    return spriteimage

In [ ]:
#generate a metadata consisting of index and label of our embedding vectors
#The Index is simply the index in our embedding matrix. The label is a string of a patient's diseases.
def create_metadata(documents, directory):
    
    match = 0
    radiologist_labels = filename_to_radiologist_labels[doc['filename']][0]
    source_labels = filename_to_source_labels[doc['filename']][0]
    if (set(radiologist_labels) == set(source_labels)):
        match = 1
    
    
    with open(directory,'w') as f:
        f.write("Index\tLabels\tFirst_label\n")
        for index,doc in enumerate(documents):
            f.write("%d\t%d\t%s\t%s\n" % (index, match, source_labels, radiologist_labels))

In [ ]:
sprite_image = create_sprite_image(documents, SPRITE_DIR)
metadata = create_metadata(documents, META_DIR)

In [ ]:
#feed embedding variables into feed into the visualizer
mu_embedding = []
sigma_embedding = []
embeddings = []
embedding_names = []

for doc in documents:
        mu_embedding.append(doc['mu'].tolist())
embeddings.append(mu_embedding)
embedding_names.append('mu')

for doc in documents:
        sigma_embedding.append(doc['sigma'].tolist())
embeddings.append(sigma_embedding)
embedding_names.append('sigma')

In [ ]:
#specify what variable you want to project, what the metadata path is (the names and classes),
#and where you save the sprites
summary_writer = tf.summary.FileWriter(VIS_DIR)
config = projector.ProjectorConfig()

for i,e in enumerate(embeddings):
    
    embedding_var = tf.Variable(e, name=embedding_names[i])
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name

    # Specify where you find the metadata
    embedding.metadata_path = 'metadata.tsv' #'metadata.tsv'

    # Specify where you find the sprite (we will create this later)
    embedding.sprite.image_path = 'sprite.png' #'sprite.png'
    embedding.sprite.single_image_dim.extend([256,256])

# Say that you want to visualise the embeddings
projector.visualize_embeddings(summary_writer, config)

In [ ]:
#Tensorboard loads the saved variable from the saved graph. 
#Initialise a session and variables, and save them in your logging directory.
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(VIS_DIR, "model1.ckpt"), 1)